In [1]:
# Reference：Build your own RAG with Mistral-7B and LangChain
# https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146

In [ ]:
%%bash
pip install git+https://github.com/huggingface/transformers.git
pip install -i https://pypi.org/simple/ bitsandbytes
pip install accelerate
pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

Process is terminated.


In [ ]:
import torch, transformers
from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  BitsAndBytesConfig,
  pipeline
)
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

#Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [ ]:
context = "2023-2024 OFFICIAL Academic Calendar Carnegie Mellon University Revised: 28 July 2023 Page 1 of 3Semester : (M-13, T-13, W-13, Th-13, F-13) Total=65 Mini-1: (M-6, T-7, W-7, Th-7, F-65) Total=31 Date Day Event August 28 M Semester & Mini-1 Classes Begin September 1 F Mini-1 add, audit & tuition adjustment drop deadline (1) September 4 M Labor Day; No Classes"
question = "When do Summer All & Summer One/Mini-5 classes begin for the 2023-2024 academic year at CMU?"
prompt = """Given the following information, answer the question concisely.
Context: {}.
Question: {}
""".format(context, question)
inputs_not_chat = tokenizer.encode_plus(prompt, return_tensors="pt")['input_ids'].to('cuda')
question_only = tokenizer.encode_plus(question, return_tensors="pt")['input_ids'].to('cuda')
generated_ids = model.generate(inputs_not_chat,
                              max_new_tokens=1000,
                              do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
answer = decoded[0].split("\n")[3].split(":")[1].replace("</s>","")
print(prompt,answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Given the following information, answer the question concisely.
Context: 2023-2024 OFFICIAL Academic Calendar Carnegie Mellon University Revised: 28 July 2023 Page 1 of 3Semester : (M-13, T-13, W-13, Th-13, F-13) Total=65 Mini-1: (M-6, T-7, W-7, Th-7, F-65) Total=31 Date Day Event August 28 M Semester & Mini-1 Classes Begin September 1 F Mini-1 add, audit & tuition adjustment drop deadline (1) September 4 M Labor Day; No Classes.
Question: When do Summer All & Summer One/Mini-5 classes begin for the 2023-2024 academic year at CMU?
  September 1


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
question_file = '/content/drive/My Drive/NLP2/FAQ_style_QA/Q_Test/Test_all.txt'
# question_file = '/content/drive/My Drive/NLP2/FAQ_style_QA/Q_Train/NLP2_Q_traning_all.txt'
model_output = '/content/drive/My Drive/NLP2/model_answer/t5_answer/model_test_answer_t5.txt'
# gold_answer = '/content/drive/My Drive/NLP2/model_answer/t5_answer/t5_gold_answer.txt'
mistral_output = '/content/drive/My Drive/NLP2/model_answer/mistral_answer/mistral_test_answer.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open(question_file, "r", encoding='utf-8') as file:
  questions = file.readlines()
with open(model_output, "r", encoding='utf-8') as file:
  outputs = file.readlines()
print(len(questions), len(outputs))
with open(mistral_output, "w", encoding='utf-8') as file:
  for i in range(len(outputs)):
    prompt = """Given the following information, answer the question concisely. You don't have to answer the full sentence. @Context: {} @Question: {}@""".format(outputs[i].strip(), questions[i].strip().replace("\n", ""))
    if "---------------" in outputs[i]:
      prompt = "@@@".format(questions[i].strip().replace("\n", ""))
    print(questions[i].strip())
    inputs = tokenizer.encode_plus(prompt, return_tensors="pt")['input_ids'].to('cuda')
    generated_ids = model.generate(inputs,
                              max_new_tokens=1000,
                              do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    print(decoded)
    answer = decoded[0].split("@")[3]
    if ":" in answer:
      answer=answer.split(":")[1].replace("</s>","")
    print(prompt, answer)
    file.write(f"{answer}\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


81 81
When did the Carnegie Technical Schools start offering bachelor's degrees?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Given the following information, answer the question concisely. You don\'t have to answer the full sentence. @Context: : History - CMU - Carnegie Mellon University Carnegie Mellon University ——— Search Search CMU  About  Andrew Carnegie A self-educated "working boy" who loved books, Andrew Carnegie emigrated from Scotland in 1848 and settled in Pittsburgh, Pa. In 1900, he donated $1 million for the creation of a technical institute for the city of Pittsburgh. @Question: When did the Carnegie Technical Schools start offering bachelor\'s degrees?@Answer: In 1910, the Carnegie Technical Schools started offering bachelor\'s degrees.</s>']
Given the following information, answer the question concisely. You don't have to answer the full sentence. @Context: : History - CMU - Carnegie Mellon University Carnegie Mellon University ——— Search Search CMU  About  Andrew Carnegie A self-educated "working boy" who loved books, Andrew Carnegie emigrated from Scotland in 1848 and settled in Pitts

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

['<s> Given the following information, answer the question concisely. You don\'t have to answer the full sentence. @Context: and LM subnetwork to make the final prediction. Context: Integrating Pretrained ASR and LM to Perform Sequence Generation for Spoken Language Understanding Siddhant Arora1, Hayato Futami2, Yosuke Kashiwagi2, Emiru Tsunoo2, Brian Yan1, Shinji Watanabe1 1Carnegie Mellon University, U.S.A. @Question: What is the main focus of "Integrating Pretrained ASR and LM to Perform Sequence Generation for Spoken Language Understanding"?@Answer: The main focus is to make a final prediction using an and LM subnetwork.</s>']
Given the following information, answer the question concisely. You don't have to answer the full sentence. @Context: and LM subnetwork to make the final prediction. Context: Integrating Pretrained ASR and LM to Perform Sequence Generation for Spoken Language Understanding Siddhant Arora1, Hayato Futami2, Yosuke Kashiwagi2, Emiru Tsunoo2, Brian Yan1, Shinji W

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Given the following information, answer the question concisely. You don\'t have to answer the full sentence. @Context: @gmail.com ABSTRACT This paper describes our submission to the Second Clarity Enhance- ment Challenge (CEC2). We introduce four major novelties: (1) we extend the state-of-the-art TF-GridNet model [1], originally designed for monaural speaker separation, for multi-channel, causal speech enhancement, and large improvements are observed. @Question: How does the "Multi-Channel Speaker Extraction with Adversarial Training" paper contribute to the Clarity ICASSP 2023 Grand Challenge?@Answer: The paper extends the state-of-the-art TF-GridNet model for multi-channel causal speech enhancement and large improvements are observed.</s>']
Given the following information, answer the question concisely. You don't have to answer the full sentence. @Context: @gmail.com ABSTRACT This paper describes our submission to the Second Clarity Enhance- ment Challenge (CEC2). We introduce

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Given the following information, answer the question concisely. You don\'t have to answer the full sentence. @Context: , a linear classifier achieves F1 score of 97.6% in recognizing the voice of 30 speakers. Autodecome uses two comple-mentary augmentations, one that manipulates the context while preserving the desired property. Autodecome uses two comple-mentary augmentations, one that manipulates the context while preserving the desired property. @Question: What is the main contribution of "Fully Unsupervised Topic Clustering of Unlabelled Spoken Audio Using Self-Supervised Representation Learning and Topic Model"?@Answer: The main contribution of "Fully Unsupervised Topic Clustering of Unlabelled Spoken Audio Using Self-Supervised Representation Learning and Topic Model" is that it achieves an F1 score of 97.6% in recognizing the voice of 30 speakers with the help of autodecome.</s>']
Given the following information, answer the question concisely. You don't have to answer the 

In [ ]:
print(outputs[1])

2023-2024 OFFICIAL Academic Calendar Carnegie Mellon University Revised: 28 July 2023 Page 1 of 3Semester : (M-13, T-13, W-13, Th-13, F-13) Total=65 Mini-1: (M-6, T-7, W-7, Th-7, F-65) Total=31 Date Day Event August 28 M Semester & Mini-1 Classes Begin September 1 F Mini-1 add, audit & tuition adjustment drop deadline (1) September 4 M Labor Day; No Classes

